In [1]:
import sem
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import seaborn as sns
from datetime import datetime

# Create our SEM campaign

#Manual
#https://simulationexecutionmanager.readthedocs.io/en/develop/

#example
#https://github.com/signetlabdei/sem/blob/master/examples/lorawan_parsing_example.py

script = 'complete-network-example'

#List of simulations I have run + their settings.



#'testing_num_of_nodes-1'. r=[2000,5000, 6300] m. (20) + 30 Periods. 6 arrs (0.143-2.031).  10 runs.


current_sim = 'testing_num_of_nodes-1'

#Saving results of simulations in their own folders

results_dir = ('sim-results/results-' + current_sim)

#lorawan is located in ns-3-unmodified folder
ns_3_dir = '../../../'


#Disabling checking if commit # matches
#overwite is to indicate if existing simulations must be overwritten or should they be used

campaign = sem.CampaignManager.new(ns_3_dir, script, results_dir,
                                   check_repo=False, overwrite=False)

# Parameter space
#################


radius_values = [2000, 5000] # [2000,5000,6300] #in m, max is 6400.

#The period in seconds to be used by periodically transmitting applications
appPeriod_values = [] 


#Number of repeats (runs) to do for averaging
runs = 2



#number of devices in simulation
num_devices = 1200



# I don't know the exact x-values used in davide's research group's paper fig 2, picking my own.
arrival_rates = list(np.logspace(start= -2, stop=1, num=14))

for index, value in enumerate(arrival_rates):
    arrival_rates[index] = np.round(arrival_rates[index], 3)

# Fig 2 only plots till 2 pkt/s, but I generated up till 10 pkt/s.
arrival_rates = arrival_rates[:-3] #they only plot till 2, dropping last 3 generated rates.


for arrival_rate in arrival_rates:

    # 1200 devices will send in 1 appPeriod 1200 packets
    # different appPeriods will generated different arrival rates (lamdba)
    # appPeriod = 1200/(lambda)

    appPeriod_values.append(np.ceil(num_devices/arrival_rate))
    print("For ", arrival_rate, " pkt/s, app period for ", num_devices, " devices must be ",  appPeriod_values[-1], "s")



#For debugging, the amount of values can be reduced here to better see what is happening
#arrival_rates = [2.031]
#appPeriod_values = [ 591]

#dictionary to map between appPeriods and arrival rates

dictionary = dict(zip(appPeriod_values, arrival_rates))

#The values that sem will use to run script
param_combinations = {
  'appPeriod': appPeriod_values,
  'nDevices' : num_devices,
  'radius' : radius_values,
  'simulationTime' : 1,
  'print' : 1
}

#The last two the sim requires but I don't want to modify them. simulationTime will be overwritten and print I just kept as true 
   

print("\n", param_combinations)

now = datetime.now().strftime("%d-%m-%Y-%H:%M:%S") # current time
print("Starting sims at " + now)
for i in range(2):
    campaign.run_missing_simulations(param_combinations, runs)

now = datetime.now().strftime("%d-%m-%Y-%H:%M:%S") # current time
print("Sims finished at " + now)
print("Simulation results saved/loaded from " + campaign.db.get_data_dir())

overall_performance_data = [] # saving metrics  captured on the entire simulation


For  0.01  pkt/s, app period for  1200  devices must be  120000.0 s
For  0.017  pkt/s, app period for  1200  devices must be  70589.0 s
For  0.029  pkt/s, app period for  1200  devices must be  41380.0 s
For  0.049  pkt/s, app period for  1200  devices must be  24490.0 s
For  0.084  pkt/s, app period for  1200  devices must be  14286.0 s
For  0.143  pkt/s, app period for  1200  devices must be  8392.0 s
For  0.242  pkt/s, app period for  1200  devices must be  4959.0 s
For  0.412  pkt/s, app period for  1200  devices must be  2913.0 s
For  0.702  pkt/s, app period for  1200  devices must be  1710.0 s
For  1.194  pkt/s, app period for  1200  devices must be  1006.0 s
For  2.031  pkt/s, app period for  1200  devices must be  591.0 s

 {'appPeriod': [120000.0, 70589.0, 41380.0, 24490.0, 14286.0, 8392.0, 4959.0, 2913.0, 1710.0, 1006.0, 591.0], 'nDevices': 1200, 'radius': [2000, 5000], 'simulationTime': 1, 'print': 1}
Starting sims at 24-05-2021-12:38:17
Sims finished at 24-05-2021-12:38:17

In [2]:
# Function to parse stdout file and append to overall_performance_data
def parse_file_for_metrics(result):

    line = result['output']['stdout'].split("\n")
    line = line[0] #removing the \n at the end by taking the first option
    overall_performance_data.extend(get_list_of_values(line, result))

    

# Function to parse file and return metrics as a list
def get_list_of_values(line, result):
    
    data = []
    

    splitted = line.split(" ")
    print(splitted)
    sent = float(splitted[0])
    received = float(splitted[1])


    #Add the info 
    data.append([ result['params']['appPeriod'], dictionary[result['params']['appPeriod']],  result['params']['radius'],
                  sent, received,
                ])
 
    return data

In [3]:
#Extract values
campaign.get_results_as_xarray(param_combinations,
                                            parse_file_for_metrics, "", runs)



#Put values into a dataframe   

list_column_names = ["appPeriod", "arrival rate", "radius",  "MACSent", "MACReceived"]

#Savings the overall data in its df
overall_temp_df = pd.DataFrame(overall_performance_data, columns = list_column_names)


#Print first 15 entries
print(overall_temp_df.head(15))


print("\nThe overall temp df should have appPeriod_values x radius_values  x runs:",
      len(appPeriod_values) * len (radius_values) * runs )
print("\n It has ", len(overall_temp_df.index), " rows")


['12000.000000', '11995.000000']
['12000.000000', '12000.000000']
['12000.000000', '12000.000000']
['12000.000000', '11990.000000']
['12000.000000', '12000.000000']
['12000.000000', '11997.000000']
['12000.000000', '12000.000000']
['12000.000000', '11999.000000']
['12000.000000', '11995.000000']
['12000.000000', '11980.000000']
['12000.000000', '11992.000000']
['12000.000000', '11962.000000']
['12000.000000', '11977.000000']
['12000.000000', '11990.000000']
['12000.000000', '11979.000000']
['12000.000000', '11982.000000']
['12000.000000', '11970.000000']
['12000.000000', '11980.000000']
['12000.000000', '11955.000000']
['12000.000000', '11972.000000']
['12000.000000', '11946.000000']
['12000.000000', '11948.000000']
['12000.000000', '11954.000000']
['12000.000000', '11963.000000']
['12000.000000', '11902.000000']
['12000.000000', '11916.000000']
['12000.000000', '11914.000000']
['12000.000000', '11916.000000']
['12000.000000', '11912.000000']
['12000.000000', '11850.000000']
['12000.00

In [4]:
metric_colums = ["PDR"]

for metric in metric_colums:
        overall_temp_df[metric] = np.nan

#Add the new columns to the list of columns
list_column_names.extend(metric_colums)

overall_temp_df["PDR"] = np.round(overall_temp_df["MACReceived"].to_numpy()/overall_temp_df["MACSent"].to_numpy()*100,2)

#Print first 15 entries
print(overall_temp_df.head(15))

    appPeriod  arrival rate  radius  MACSent  MACReceived     PDR
0    120000.0         0.010    2000  12000.0      11995.0   99.96
1    120000.0         0.010    2000  12000.0      12000.0  100.00
2    120000.0         0.010    5000  12000.0      12000.0  100.00
3    120000.0         0.010    5000  12000.0      11990.0   99.92
4     70589.0         0.017    2000  12000.0      12000.0  100.00
5     70589.0         0.017    2000  12000.0      11997.0   99.98
6     70589.0         0.017    5000  12000.0      12000.0  100.00
7     70589.0         0.017    5000  12000.0      11999.0   99.99
8     41380.0         0.029    2000  12000.0      11995.0   99.96
9     41380.0         0.029    2000  12000.0      11980.0   99.83
10    41380.0         0.029    5000  12000.0      11992.0   99.93
11    41380.0         0.029    5000  12000.0      11962.0   99.68
12    24490.0         0.049    2000  12000.0      11977.0   99.81
13    24490.0         0.049    2000  12000.0      11990.0   99.92
14    2449

### Averaging  ###

In [5]:
#Going to average over the number of runs
avg_data = []

for appPeriod in appPeriod_values:
    
    for rad in radius_values:
        

                
        mask = (overall_temp_df['appPeriod'] == appPeriod) & (overall_temp_df['radius'] == rad) 



        averages = overall_temp_df[mask].mean().round(2).to_numpy() 
        avg_data.append(averages) # append to growing list that will be used to create df



overall_df = pd.DataFrame(avg_data, columns = list_column_names)  



In [6]:

(overall_df[["appPeriod", "radius","PDR"]])

,appPeriod,radius,PDR
0,120000.0,2000.0,99.98
1,120000.0,5000.0,99.96
2,70589.0,2000.0,99.99
3,70589.0,5000.0,100.00
4,41380.0,2000.0,99.90
5,41380.0,5000.0,99.80
6,24490.0,2000.0,99.86
7,24490.0,5000.0,99.84
8,14286.0,2000.0,99.79
9,14286.0,5000.0,99.70


In [7]:
#Need to make the folder dfs yourself

overall_df.to_csv('dfs/overall_'+   current_sim +'.txt', sep='\t')


#saving the sim we are currently busy with, allows for easier running of graph notebook.
f = open("current_sim.txt", "w")
f.writelines([current_sim])
f.close()
    
now = datetime.now().strftime("%d-%m-%Y-%H:%M:%S") # current time    
print("Saving completed." + now)

Saving completed.24-05-2021-12:38:18
